# Initial Development log

## Content

1. [Observations : Whalin validation [2024-01-08]](#log_bsnq_v001_1)

## Attempting

- Set up initial working code
- Verify against Whalina nd Ship wake sims

## List of Work


---
---

<a id = 'log_bsnq_v001_1' />

## Observations : Whalin validation [2024-01-08]

Basic Julia code was complete along with 

- All Bsnq terms
- Fourier3 wavemaking
- Sponge layer (hard coded)

I executed the Whalin shoal test case to validate the Julia code against the FORTRAN code.

The key differences between the two codes at the moment are

|  | Bsnq FORTRAN | Bsnq Julia |
| :--- | :--- | :--- |
| Time-stepping | RK4 | Theta Method (0.5) |
|  | OpenMP parallel | Serial |
| NL Solver | BiCGStab | Newton (Backtracking) |

---

### Comparison of results - Whalin Shoal

- I executed the Whalin shoal case in both Bsnq-Julia and Bsnq-FORTRAN
- Parameters are as follows

|  | Bsnq FORTRAN | Bsnq Julia |
| :--- | :--- | :--- |
| Plot | Red | Blue |
| Mesh | Triangles | Quads |
| dx = dy | 0.1016 | 0.1016 |
| dt | 0.04 | 0.04 |
| NL Solver Tolerance | absTol = 1e-5 | ftol = 1e-5 |

| Surface elevation eta |
| --- |
| <img width="100%" src="./log_v001/c01/jul_1016_dt04_fort_1016_dt04.png"> |


- We clearly observe a dissipation in the surface elevation in Bsnq-Julia. 
- **I believe this is due to the time-stepping.** This is confirmed in the following tests.
- The following is the comparison if I reduce the time-step for Bsnq-Julia

|  | Bsnq FORTRAN | Bsnq Julia |
| :--- | :--- | :--- |
| Plot | Red | Blue |
| Mesh | Triangles | Quads |
| dx = dy | 0.1016 | 0.1016 |
| dt | 0.04 | 0.02 |
| NL Solver Tolerance | absTol = 1e-5 | ftol = 1e-5 |

| Surface elevation eta |
| --- |
| <img width="100%" src="./log_v001/c01/jul_1016_dt02_fort_1016_dt04.png"> |

- For run-time, refer to [link](#log_bsnq_v001_1_3)

---

### Checking influence of time-step, tolerance and Integration Points

#### Test 1: Mesh Triangle vs Rectangle

|  | Bsnq Julia | Bsnq Julia |
| :--- | :--- | :--- |
| Plot | Red | Blue |
| Mesh | Triangles | Quads |
| dx = dy | 0.2032 | 0.2032 |
| dt | 0.08 | 0.08 |
| NL Solver Tolerance | absTol = 1e-5 | ftol = 1e-5 |

No visible difference


#### Test 2: Number of integration points

|  | Bsnq Julia | Bsnq Julia |
| :--- | :--- | :--- |
| Plot | Red | Blue |
| Mesh | Quads | Quads |
| dx = dy | 0.2032 | 0.2032 |
| dt | 0.08 | 0.08 |
| NL Solver Tolerance | absTol = 1e-5 | ftol = 1e-5 |
| **Measure = ** | 8 | 4 (default) |

No visible difference

| Surface elevation eta|
| --- |
| <img width="100%" src="./log_v001/c01/jul_2032_dt08_tol5_meas4_jul_2032_dt08_tol5_meas8.png"> |


#### Test 3: Solver tolerence

|  | Bsnq Julia | Bsnq Julia |
| :--- | :--- | :--- |
| Plot | Red | Blue |
| Mesh | Quads | Quads |
| dx = dy | 0.2032 | 0.2032 |
| dt | 0.08 | 0.08 |
| NL Solver Tolerance | absTol = 1e-5 | ftol = 1e-8 |

Minimal difference

| Surface elevation eta|
| --- |
| <img width="100%" src="./log_v001/c01/jul_2032_dt08_tol5_jul_2032_dt08_tol8.png"> |


#### Test 4: Time-step

|  | Bsnq Julia | Bsnq Julia |
| :--- | :--- | :--- |
| Mesh | Quads | Quads |
| dx = dy | 0.2032 | 0.2032 |
| NL Solver Tolerance | absTol = 1e-5 | ftol = 1e-8 |

Significant difference

| Surface elevation eta|
| --- |
| Red: dt = 0.04s, Blue: dt = 0.08s |
| <img width="100%" src="./log_v001/c01/jul_2032_dt08_tol5_jul_2032_dt04_tol5.png"> |
|  |
| Red: dt = 0.02s Blue: dt = 0.08s |
| <img width="100%" src="./log_v001/c01/jul_2032_dt08_tol5_jul_2032_dt02_tol5.png"> |
|  |
| Red: dt = 0.02s Blue: dt = 0.04s |
| <img width="100%" src="./log_v001/c01/jul_2032_dt04_tol5_jul_2032_dt02_tol5.png"> |

---

<a id = 'log_bsnq_v001_1_3' />

### Speed comparison

|  | Bsnq FORTRAN | Bsnq Julia |
| :--- | :--- | :--- |
| Time-stepping | RK4 | Theta Method (0.5) |
|  | OpenMP parallel | Serial |
| NL Solver | BiCGStab | Newton (Backtracking) |
| NL Solver Tolerance | absTol = 1e-5 | ftol = 1e-5 |

Simulation over 40s.<br>
All run-time values are in seconds. All tests were run on Sherlock, unless specified.

|  | Bsnq-Julia (Serial) | Bsnq-Fortran (Serial) | Bsnq-Fortran (OpenMP Cores) |
| :--- | :--- | :--- | :--- |
| dx=0.2032, dt=0.08 | 1521 Sherlock | 153 Sherlock | 89 (4) Sherlock | 
| dx=0.1016, dt=0.04 | 9468 Sherlock <br> | 1486 Sherlock <br> 1797 Watson | 1147 (4) Sherlock <br> 725 (4) Watson | 
| dx=0.1016, dt=0.02 | 18684 |   |  | 

I could not compile the bsnq code on Sherlock. Probably thats why the OpenMP performance is so poor.

---

---
---

## References

1. Sørensen, O. R., Schäffer, H. A., & Sørensen, L. S. (2004). Boussinesq-type modelling using an unstructured finite element technique. Coastal Engineering, 50(4), 181–198. [DOI](https://doi.org/10.1016/j.coastaleng.2003.10.005)

1. Agarwal, S., Sriram, V., & Murali, K. (2019). Modelling Wave Interaction with Porous Structures Using Boussinesq Equations. In Proceedings of the Fourth International Conference in Ocean Engineering (ICOE2018) (pp. 573–583). <https://doi.org/10.1007/978-981-13-3119-0_35>

1. Dresback, K. M., Kolar, R. L., & Dietrich, J. C. (2004). A 2D implicit time-marching algorithm for shallow water models based on the generalized wave continuity equation. International Journal for Numerical Methods in Fluids, 45(3), 253–274. https://doi.org/10.1002/fld.697

1. Lee, S.-J., Yates, G. T., & Wu, T. Y. (1989). Experiments and analyses of upstream-advancing solitary waves generated by moving disturbances. Journal of Fluid Mechanics, 199(1), 569. https://doi.org/10.1017/S0022112089000492